Importing libraries

In [21]:
import pandas as pd
import numpy as np

Loading the data

In [ ]:
df = pd.read_csv('data/autos.csv', encoding='latin-1') 

Declaring functions

In [23]:
def drop_index(df,index):
    # Dropping specified indexes and reporting on a total % change in dataset size
    df = df.drop(index)
    len_index = len(index)
    percent = np.round(100*len_index/len(df),2)
    print(f"{len_index} observations dropped ({percent}% of the current form of the dataset)")
    return df

def dummify(df, col):
    # Creating dummy variables and reporting on the created amount
    dummies = pd.get_dummies(df[col], prefix=col, dummy_na=False, dtype=int)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(col, axis=1)
    print(f"{dummies.shape[1]} dummies created from {col} variable")
    return df

Calculating the age of a car as a difference between the time the ad was scraped and the registration year. 

Dropping all very old cars (>30 years old), new cars (current and later model years) and errors.

In [24]:
df['dateCrawled'] = pd.to_datetime(df['dateCrawled'], errors='coerce')
df['age'] = df['dateCrawled'].dt.year - df['yearOfRegistration']
wrong_age_index = df[(df['age']>30) | (df['age']<1)].index
df = drop_index(df, wrong_age_index)
df = df.drop(['dateCrawled','yearOfRegistration','monthOfRegistration'], axis=1)

31570 observations dropped (9.29% of the current form of the dataset)


Dropping all damaged or with unknown mechanical state cars

In [25]:
print(df['notRepairedDamage'].value_counts())
broken_car_index = df[(df['notRepairedDamage']=='ja') | (df['notRepairedDamage'].isna())].index
df = drop_index(df, broken_car_index)
df = df.drop('notRepairedDamage', axis=1)

notRepairedDamage
nein    247766
ja       33218
Name: count, dtype: int64
92192 observations dropped (37.21% of the current form of the dataset)


Dropping redundant variables:
- seller: only 1 observation has a different value than the rest
- name: includes information that other variables provide, too granular in its current form
- model: too granular in its current form, vehicleType and brand should be enough to model a price of a car
- offerType: only 3 observations have a different value than the rest
- abtest: irrelevant to the study
- postalCode: irrelevant to the study
- lastSeen: irrelevant to the study
- dateCreated: irrelevant to the study
- nrOfPictures: all values are equal to 0

In [26]:
redundant_cols = ['seller', 'name', 'model', 'offerType', 'abtest', 'postalCode', 'lastSeen', 'dateCreated', 'nrOfPictures']
for col in redundant_cols:
    df = df.drop(col, axis=1)

Dropping remaining missing values

In [27]:
na_mask = df.isnull()
na_rows = na_mask.any(axis=1)
na_index = df.index[na_rows]
df = drop_index(df, na_index)

13026 observations dropped (5.55% of the current form of the dataset)


Final set of variables before encoding

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 234740 entries, 3 to 371527
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   price        234740 non-null  int64 
 1   vehicleType  234740 non-null  object
 2   gearbox      234740 non-null  object
 3   powerPS      234740 non-null  int64 
 4   kilometer    234740 non-null  int64 
 5   fuelType     234740 non-null  object
 6   brand        234740 non-null  object
 7   age          234740 non-null  int64 
dtypes: int64(4), object(4)
memory usage: 16.1+ MB


Descriptive statistics of the variables

In [29]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,234740.0,8280.532108,296403.071630,0.0,1800.0,4250.0,9200.0,99999999.0
powerPS,234740.0,128.990726,151.805864,0.0,80.0,116.0,160.0,20000.0
kilometer,234740.0,123009.457272,40197.103456,5000.0,100000.0,150000.0,150000.0,150000.0
age,234740.0,12.065558,5.626494,1.0,8.0,12.0,16.0,30.0


Dropping all cars with price equal to 0 or higher than 200000 euros

In [30]:
price_index = df[(df['price']==0) | (df['price']>200000)].index
df = drop_index(df, price_index)

2354 observations dropped (1.01% of the current form of the dataset)


Dropping all cars with 0 or over 700 PS

In [31]:
ps_index = df[(df['powerPS']>700) | (df['powerPS'])==0].index
df = drop_index(df, ps_index)

7413 observations dropped (3.3% of the current form of the dataset)


Dropping cars not powered by diesel or regular combustion engines

In [32]:
fuel_index = df[~df['fuelType'].isin(['benzin', 'diesel'])].index
df = drop_index(df, fuel_index)

4082 observations dropped (1.85% of the current form of the dataset)


Encoding binary variables

In [33]:
df['gearbox'] = df['gearbox'].map({'manuell':1,'automatik':0}).astype(int)
df['gearbox'].value_counts()

gearbox
1    168203
0     52688
Name: count, dtype: int64

In [34]:
df['fuelType'] = df['fuelType'].map({'benzin':0,'diesel':1}).astype(int)
df['fuelType'].value_counts()

fuelType
0    143111
1     77780
Name: count, dtype: int64

Final descriptive statistics of numerical variables

In [35]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
price,220891.0,7286.557153,8819.513878,1.0,1950.0,4499.0,9500.0,200000.0
gearbox,220891.0,0.761475,0.426183,0.0,1.0,1.0,1.0,1.0
powerPS,220891.0,132.866464,150.771325,1.0,87.0,118.0,160.0,20000.0
kilometer,220891.0,122424.996944,40360.250517,5000.0,100000.0,150000.0,150000.0,150000.0
fuelType,220891.0,0.352119,0.477632,0.0,0.0,0.0,1.0,1.0
age,220891.0,11.919245,5.613558,1.0,8.0,12.0,16.0,30.0


Encoding categorical variables

In [36]:
df = dummify(df, 'vehicleType')

8 dummies created from vehicleType variable


In [37]:
df = dummify(df, 'brand')

40 dummies created from brand variable


Saving the cleaned dataset

In [ ]:
df.to_csv('data/autos_clean.csv', index=None)